In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# /Users/chriswesterman/Library/Jupyter/nbextensions/snippets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import beatthebookies.data as btb
import sqlite3

pd.set_option('display.max_columns', 500)


# Exploratory notebook

In [3]:
df = btb.get_csv_data()

get_csv_data 161.94


In [4]:
df.head()

,season,date,home_team,away_team,home_team_goal,away_team_goal,FTR,HTHG,HTAG,HTR,Referee,home_shots,away_shots,home_shots_targ,away_shots_targ,home_fouls,away_fouls,HC,AC,home_yel,away_yel,home_red,away_red,WHH,WHD,WHA,H_ATT,H_MID,H_DEF,H_OVR,A_ATT,A_MID,A_DEF,A_OVR,stage,home_w,draw,away_w,home_t_home_goals,home_t_home_goals_against,home_t_home_wins,home_t_home_losses,home_t_home_draws,away_t_away_goals,away_t_away_goals_against,away_t_away_wins,away_t_away_losses,away_t_away_draws,home_t_total_goals,home_t_total_goals_against,home_t_total_wins,home_t_total_losses,home_t_total_draws,away_t_total_goals,away_t_total_goals_against,away_t_total_wins,away_t_total_losses,away_t_total_draws
0,2008/2009,2008-08-16,Arsenal,West Brom,1.0,0.0,H,1.0,0.0,H,H Webb,24.0,5.0,14.0,4.0,11.0,8.0,7.0,5.0,0.0,0.0,0.0,0.0,1.17,5.5,12.00,85.0,83.0,83.0,81.0,68.0,71.0,69.0,71.0,1,1,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0
1,2008/2009,2008-08-16,Bolton,Stoke,3.0,1.0,H,3.0,0.0,H,C Foy,14.0,8.0,8.0,2.0,13.0,12.0,4.0,3.0,1.0,2.0,0.0,0.0,1.80,3.1,4.00,75.0,75.0,72.0,74.0,75.0,68.0,67.0,70.0,1,1,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0
2,2008/2009,2008-08-16,Everton,Blackburn,2.0,3.0,A,1.0,1.0,D,A Marriner,10.0,15.0,5.0,11.0,11.0,9.0,3.0,5.0,2.0,2.0,0.0,0.0,1.95,3.1,3.50,82.0,75.0,78.0,78.0,78.0,76.0,76.0,76.0,1,0,0,1,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0
3,2008/2009,2008-08-16,Hull,Fulham,2.0,1.0,H,1.0,1.0,D,P Walton,11.0,12.0,6.0,6.0,10.0,9.0,5.0,6.0,3.0,0.0,0.0,0.0,2.50,2.9,2.62,70.0,68.0,68.0,71.0,80.0,74.0,72.0,74.0,1,1,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0
4,2008/2009,2008-08-16,Middlesbrough,Tottenham,2.0,1.0,H,0.0,0.0,D,M Atkinson,14.0,8.0,10.0,5.0,11.0,12.0,7.0,9.0,1.0,2.0,0.0,0.0,2.90,3.2,2.15,80.0,75.0,73.0,74.0,80.0,80.0,79.0,79.0,1,1,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0


In [7]:
df.to_csv("../data/tempfile.csv", index=False, encoding='utf-8-sig')

In [145]:
4560 / 380

12.0

In [22]:
import itertools
lst = range(1,39)
season = list(itertools.chain.from_iterable(itertools.repeat(x, 10) for x in lst))

In [23]:
season

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,


In [26]:
full = list(itertools.chain.from_iterable(itertools.repeat(season, 12)))

In [27]:
len(full)

4560

In [28]:
df['stage'] = full

In [30]:
df.tail()

,season,date,home_team,away_team,home_team_goal,away_team_goal,FTR,HTHG,HTAG,HTR,Referee,home_shots,away_shots,home_shots_targ,away_shots_targ,home_fouls,away_fouls,HC,AC,home_yel,away_yel,home_red,away_red,WHH,WHD,WHA,H_ATT,H_MID,H_DEF,H_OVR,A_ATT,A_MID,A_DEF,A_OVR,home_w,draw,away_w,home_t_home_goals,home_t_home_goals_against,home_t_home_wins,home_t_home_losses,home_t_home_draws,away_t_away_goals,away_t_away_goals_against,away_t_away_wins,away_t_away_losses,away_t_away_draws,stage
4552,2019/2020,2020-07-26,Chelsea,Wolves,2.0,0.0,H,2.0,0.0,H,S Attwell,11.0,5.0,3.0,1.0,10.0,16.0,3.0,3.0,2.0,3.0,0.0,0.0,1.83,3.7,4.33,80.0,84.0,81.0,81.0,79.0,79.0,78.0,78.0,1,0,0,28.0,16.0,10,5,3,24.0,19.0,7,4,7,38
4551,2019/2020,2020-07-26,Burnley,Brighton,1.0,2.0,A,1.0,1.0,D,J Moss,13.0,14.0,2.0,4.0,11.0,11.0,6.0,7.0,3.0,0.0,0.0,0.0,2.30,3.4,3.10,77.0,76.0,76.0,76.0,75.0,76.0,76.0,76.0,0,0,1,23.0,21.0,8,6,4,17.0,26.0,3,8,7,38
4550,2019/2020,2020-07-26,Arsenal,Watford,3.0,2.0,H,3.0,1.0,H,M Dean,13.0,19.0,5.0,6.0,9.0,12.0,4.0,8.0,3.0,3.0,0.0,0.0,1.91,3.9,3.75,84.0,82.0,78.0,80.0,76.0,78.0,75.0,77.0,1,0,0,33.0,22.0,9,3,6,12.0,34.0,2,12,4,38
4553,2019/2020,2020-07-26,Crystal Palace,Tottenham,1.0,1.0,D,0.0,1.0,A,A Marriner,13.0,7.0,2.0,2.0,11.0,13.0,7.0,2.0,3.0,2.0,0.0,0.0,6.50,4.2,1.52,73.0,77.0,76.0,77.0,85.0,82.0,82.0,82.0,0,1,0,14.0,19.0,6,8,4,24.0,29.0,4,7,7,38
4559,2019/2020,2020-07-26,West Ham,Aston Villa,1.0,1.0,D,0.0,0.0,D,M Oliver,10.0,13.0,1.0,4.0,16.0,13.0,0.0,7.0,2.0,1.0,0.0,0.0,3.00,3.6,2.30,82.0,78.0,73.0,78.0,77.0,75.0,75.0,76.0,0,1,0,29.0,32.0,6,9,3,18.0,36.0,2,12,4,38


In [16]:
for i in range(12):
    season.append(season)

In [17]:
len(season)


392

In [21]:
season[0]

1

In [18]:
flat = []
for sublist in season[380:]:
    for item in sublist:
        flat.append(item)

In [19]:
len(flat)

4704

In [11]:
path = '../data/fifarank.csv'

In [12]:
df = pd.read_csv(path)

In [16]:
df.Team.unique()

array(['Arsenal', 'Manchester United', 'Chelsea', 'Liverpool',
       'Newcastle United', 'Middlesbrough', 'Fulham', 'Tottenham Hotspur',
       'Manchester City', 'Birmingham City', 'Aston Villa',
       'Charlton Athletic', 'Bolton Wanderers', 'Blackburn Rovers',
       'Southampton', 'Portsmouth', 'Everton', 'West Bromwich',
       'Norwich City', 'Crystal Palace', 'West Ham United',
       'Wigan Athletic', 'Sunderland', 'Sheffield United', 'Reading FC',
       'Watford', 'Chelsea FC', 'Arsenal FC', 'Derby County', 'Hull City',
       'Stoke City', 'Burnley', 'Wolverhampton Wanderers',
       'West Bromwich Albion', 'Blackpool', 'Queens Park Rangers',
       'Swansea City', 'Reading', 'Cardiff City', 'Leicester City',
       'Bournemouth', 'Brighton & Hove Albion', 'Huddersfield Town',
       'AFC Bournemouth'], dtype=object)

In [5]:
matches = btb.get_data()

get_data 9.0


In [7]:
seasons = ['06','07','08','09','10','11','12','13','14','15','16','17','18','19','20']
for year in seasons:
    print(f'20{year}/20{str(int(year)+1).zfill(2)}')

2006/2007
2007/2008
2008/2009
2009/2010
2010/2011
2011/2012
2012/2013
2013/2014
2014/2015
2015/2016
2016/2017
2017/2018
2018/2019
2019/2020
2020/2021


In [6]:
matches.tail()

,id,season,stage,date,home_team,away_team,home_team_goal,away_team_goal,home_w,away_w,draw,home_t_home_goals,home_t_home_goals_against,home_t_home_wins,home_t_home_losses,home_t_home_draws,away_t_away_goals,away_t_away_goals_against,away_t_away_wins,away_t_away_losses,away_t_away_draws,home_t_total_goals,home_t_total_goals_against,home_t_total_wins,home_t_total_losses,home_t_total_draws,away_t_total_goals,away_t_total_goals_against,away_t_total_wins,away_t_total_losses,away_t_total_draws,bups_x,bupp_x,ccp_x,ccc_x,ccs_x,dp_x,da_x,dtw_x,bups_y,bupp_y,ccp_y,ccc_y,ccs_y,dp_y,da_y,dtw_y
375,4701,2015/2016,38,2016-05-15 00:00:00,Everton,Norwich City,3,0,1,0,0,32,30,5,8,5,13,34,3,13,2,56,55,10,13,14,39,64,9,21,7,43,40,39,33,63,52,58,59,45,55,46,54,47,38,39,51
376,4700,2015/2016,38,2016-05-15 00:00:00,Chelsea,Leicester City,1,1,0,0,1,31,29,5,5,8,32,17,11,2,5,58,52,12,12,13,67,35,23,3,11,67,36,41,34,44,39,41,46,63,60,47,64,46,58,65,55
377,4699,2015/2016,38,2016-05-15 00:00:00,Arsenal,Aston Villa,4,0,1,0,0,27,11,11,3,4,13,37,1,14,3,61,36,19,7,11,27,72,3,26,8,59,30,28,44,46,51,44,52,63,54,60,48,38,35,44,54
378,4704,2015/2016,38,2016-05-15 00:00:00,Southampton,Crystal Palace,4,1,1,0,0,35,21,10,5,3,19,24,5,7,6,55,40,17,11,9,38,47,11,17,9,52,46,42,54,30,50,65,44,61,67,67,69,50,36,34,41
379,4702,2015/2016,38,2016-05-17 00:00:00,Manchester United,Bournemouth,3,1,1,0,0,24,8,11,2,5,21,30,6,8,4,46,34,18,10,9,44,64,11,17,9,38,44,49,44,40,54,53,56,47,47,49,45,47,48,50,62


In [ ]:
sql_path = '../data/'
database = sql_path + 'database.sqlite'
conn = sqlite3.connect(database)

In [ ]:
df = pd.read_sql("""SELECT m.id,
                        m.season, m.stage, m.date,
                        ht.team_long_name as home_team, at.team_long_name as away_team, m.home_team_goal,
                        m.away_team_goal, m.home_team_api_id as home_api, m.away_team_api_id as away_api
                        FROM Match as m
                        LEFT JOIN Team AS ht on ht.team_api_id = m.home_team_api_id
                        LEFT JOIN Team AS at on at.team_api_id = m.away_team_api_id
                        WHERE league_id = :league AND season != :season
                        ;""", conn, params={"league":1729, "season":'2008/2009'})

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
stats2 = pd.read_sql("""Select * from Team_Attributes""", conn)
stats2.head()

In [ ]:
stats2.buildUpPlayPositionClass.unique()

In [ ]:
h_stats = pd.read_sql("""Select * from Team_Attributes""", conn)
a_stats = pd.read_sql("""Select * from Team_Attributes""", conn)
h_stats['date'] = pd.to_datetime(h_stats['date'])
h_stats['year'] = pd.DatetimeIndex(h_stats['date']).year
season_maps = {2010: '2009/2010', 2011: '2010/2011', 2012:'2011/2012',
             2013:'2013/2014', 2014: '2014/2015', 2015:'2015/2016'}
h_stats['season'] = h_stats['year'].map(season_maps)
h_stats = h_stats[['season', 'team_api_id', 'buildUpPlaySpeed']]

a_stats['date'] = pd.to_datetime(a_stats['date'])
a_stats['year'] = pd.DatetimeIndex(a_stats['date']).year
season_maps = {2010: '2009/2010', 2011: '2010/2011', 2012:'2011/2012',
             2013:'2013/2014', 2014: '2014/2015', 2015:'2015/2016'}
a_stats['season'] = a_stats['year'].map(season_maps)
a_stats = a_stats[['season', 'team_api_id', 'buildUpPlaySpeed']]


In [ ]:
h_stats.head()

In [ ]:
h_stats[h_stats['team_api_id'] == 10260 ]

In [ ]:
a_stats[a_stats['team_api_id'] == 8655]

In [ ]:
df = pd.merge(df, h_stats, how='left', left_on=['season', 'home_api'], right_on=['season', 'team_api_id'])

In [ ]:
df = df.rename(columns={'buildUpPlaySpeed': 'home speed'})

In [ ]:
df.head()

In [ ]:
df = pd.merge(df, a_stats, how='left', left_on=['season', 'away_api'], right_on=['season', 'team_api_id'])

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df[df['home_api'] == 10260]

In [ ]:
columns = [['buildUpPlaySpeed', 'buildUpPlayDribbling', 'buildUpPlayPassing', 'chanceCreationPassing', 
            'chanceCreationCrossing','chanceCreationShooting','defencePressure','defenceAggression','defenceTeamWidth']]

In [ ]:
stats.date.value_counts()

In [ ]:
mock_score = pd.DataFrame([['A', 1, 90], ['A', 2, 80], ['A',3,100], ['B', 1, 95], ['B',2,85]], columns = ['year', 'team', 'stat'])

In [ ]:
mock = pd.DataFrame([['A',1,1],
                    ['A',2,2],
                    ['A',1,3],
                    ['A',2,4],
                    ['A',1,5],
                    ['B',1,6],
                    ['B',2,7],
                    ['B',1,8],
                    ['B',2,9],
                    ['B',1,10]], columns=['year','team','score'])

In [ ]:
mock, mock_score

In [ ]:
test = pd.merge(mock, mock_score, on=['year', 'team'])

In [ ]:
test